<a href="https://colab.research.google.com/github/TiagoBruno00/Economia_en_Sympy/blob/main/MCE_Intermezzo_14_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crecimiento endógeno con capital público

En la siguiente notebook se desarrollan las condiciones de Euler que caracterizan al modelo de crecimiento endógeno con capital público y se analiza la estabilidad del sistema en el largo plazo.

El modelo propone externalidades entre las distintas $N$ firmas y el gobierno:

\begin{equation}
Z(t) = z_{0} K_{G}^{1 - \alpha}{\left(t \right)}
\end{equation}

donde $z_0$ es un escalar y $K_G$ es el stock de capital público. La función de producción de la firma $i$ es:

\begin{equation}
Y_i(t) = Z(t) K_{i}^{\alpha}{\left(t \right)}  L_{i}^{1 - \alpha}{\left(t \right)}
\end{equation}

A continuación, en primer lugar se resuelve el problema de la firma y en segundo, el problema del consumidor. Finalmente, se evalúa la estabilidad del modelo.


In [ ]:
import sympy as sp
from sympy.assumptions import global_assumptions
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Símbolos

# Parámetros (depreciaciones e impuestos)
t = sp.symbols('t', real=True, positive=True) # tiempo
N_val = 11 # firmas
alpha = sp.symbols('alpha', domain=(0,1)) # RCE
t_y = sp.symbols('t_y', domain=(0,1)) # impuesto al producto
global_assumptions.add(t_y > 0)
global_assumptions.add(t_y < 1)
delta_k = sp.symbols('delta_k', domain=(0,1)) # depreciación del capital privado
delta_g = sp.symbols('delta_g', domain=(0,1)) # depreciación del capital publico
sigma = sp.symbols('sigma', real=True, positive=True) # elasticidad sustitución
t_c = sp.Function(f't_c',  image=(0,1))(t) # impuesto al consumo
global_assumptions.add(t_c > 0)
global_assumptions.add(t_c < 1)
rho = sp.symbols('rho', real=True, positive=True) # factor de descuento intertemporal

# Factores de Producción
K_i = [sp.Function(f'K_{i}')(t) for i in range(N_val)]
L_i = [sp.Function(f'L_{i}')(t) for i in range(N_val)]
z_0 = sp.symbols('z_0', real=True) # escalar de externalidades
Z_0 = sp.symbols('Z_0', real=True) # es z_0 * L_0 ^(1-alpha)
L_0 = sp.symbols('L_0', real=True) # cantidad de trabajo (vi)
K = sp.Function(f'K')(t) # capital privado agregado
K_G = sp.Function('K_G')(t) # capital público
Z = z_0*K_G**(1-alpha) # externalidades del capital público

# Consumo e Inversión
C = sp.Function(f'C')(t)
A = sp.Function(f'A')(t) # activos financieros
I_i = [sp.Function(f'I_{i}')(t) for i in range(N_val)]
T = sp.Function(f'T')(t) # impuesto de suma fija

# Retribuciones de Factores
w = sp.Function(f'w')(t)
r = sp.Function(f'r')(t)
R = sp.Function(f'R')(t)

# Definición de proporciones (linearización)
g = sp.symbols('g', real=True, domain=(0,t_y)) # participación del consumo público en el ingreso nacional
global_assumptions.add(g > 0)
global_assumptions.add(g < t_y)
theta = sp.Function('theta')(t) # proporción consumo-capital privado: theta = sp.Eq(theta, C/K)
k = sp.Function('k')(t) # proporción capital privado-capital publico: k = sp.Eq(k, K/K_G)
gamma_ee = sp.symbols('gamma^*', real=True)

# El problema de la firma

La función objetivo de la firma $i$ es:

\begin{equation}
V_i(0) = \int\limits_{0}^{\infty} \left(Z_{0} \left(1 - t_{y}\right) K_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} L_{i}^{1 - \alpha}{\left(t \right)} - I_{i}{\left(t \right)} - L_{i}{\left(t \right)} w{\left(t \right)}\right) e^{- R{\left(t \right)}}\, dt
\end{equation}

y la acumulación del capital:
\begin{equation}
\dot{K}_i = I_{i}{\left(t \right)} - \delta_{k} K_{i}{\left(t \right)}
\end{equation}

In [ ]:
# Función de Producción
Y_i = [Z*(K_i[i])**(alpha)*(L_i[i])**(1-alpha) for i in range(N_val)]

In [ ]:
# Problema de la Firma
V_i = [sp.Integral(((1-t_y)*Y_i[i]-w*L_i[i]-I_i[i])*sp.exp(-R), (t, 0, sp.oo)) for i in range(N_val)]

In [ ]:
# Acumulación de capital
K_dt = [sp.Eq(sp.diff(K_i[i],t), I_i[i] - delta_k*K_i[i])for i in range(N_val)]

Se plantea el siguiente Hamiltoniano:

\begin{equation}
H_i = \left(z_{0} \left(1 - t_{y}\right) K_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} L_{i}^{1 - \alpha}{\left(t \right)} - I_{i}{\left(t \right)} - L_{i}{\left(t \right)} w{\left(t \right)}\right) e^{- R{\left(t \right)}} + \left(I_{i}{\left(t \right)}- \delta_{k} K_{i}{\left(t \right)}\right) \mu{\left(t \right)}
\end{equation}

donde $R(t)= \int r(t)\, \partial t$

In [ ]:
# Hamiltoniano
mu = sp.Function('mu')(t) # multiplicador
H0 = [((1-t_y)*Y_i[i]-w*L_i[i]-I_i[i])*sp.exp(-R) + mu *(K_dt[i].rhs) for i in range(N_val)]

In [ ]:
# CPO de las firmas
CPO_L_i = []
CPO_I_i = []
CPO_multiplicador_i = []
for i in range(N_val):
  CPO_L = sp.Eq(sp.diff(H0[i], L_i[i]), 0)
  CPO_I = sp.Eq(sp.diff(H0[i], I_i[i]), 0)
  CPO_mul= sp.Eq(sp.Derivative(mu, t), -sp.diff(H0[i], K_i[i]))
  CPO_L_i.append(CPO_L)
  CPO_I_i.append(CPO_I)
  CPO_multiplicador_i.append(CPO_mul)

In [ ]:
# PMgL
w_i = []
for i in range(N_val):
  w_j = sp.solve(CPO_L_i[i], w, dict=True)[0][w]
  w_i.append(w_j)

In [ ]:
# PMgK
CPO_I_i[10] # y mu punto = -exp(-R)*r

Eq(mu(t) - exp(-R(t)), 0)

In [ ]:
CPO_multiplicador_i[0] # mu punto

Eq(Derivative(mu(t), t), -alpha*z_0*(1 - t_y)*K_0(t)**alpha*K_G(t)**(1 - alpha)*L_0(t)**(1 - alpha)*exp(-R(t))/K_0(t) + delta_k*mu(t))

In [ ]:
r_i = []
for i in range(N_val):
  r_j = sp.simplify(sp.solve(sp.Eq(-sp.exp(-R)*r, CPO_multiplicador_i[i].rhs.subs(mu, sp.exp(-R))), r)[0]) #recordar que dR/dt= r y mu = exp(-R)
  r_i.append(r_j)

In [ ]:
# Intento de Agregacion --> no se puede, solo a mano
#Y, Ka, L_0 = sp.symbols('Y(t) K(t) L_0(t)')
#sp.solve([sp.Eq(Y, sum(Y_i)),
#sp.Eq(Ka, sum(K_i)),
#sp.Eq(L_0, sum(L_i))],[Y, Ka, L_0], dict=True)[0][Y]

Se resuelven las condiciones del hamiltoniano y se obtienen las condciciones de las productividades marginales del capital y el trabajo para cada firma:

\begin{align}
w(t) &= z_{0} \left(\alpha t_{y} - \alpha - t_{y} + 1\right) K_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} L_{i}^{- \alpha}{\left(t \right)} \\
r(t) &= \frac{- z_{0} \alpha t_{y} K_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} L_{i}^{1 - \alpha}{\left(t \right)} + z_{0} \alpha K_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} L_{i}^{1 - \alpha}{\left(t \right)} - \delta_{k} K_{i}{\left(t \right)}}{K_{i}{\left(t \right)}}
\end{align}

equivalentes a:

\begin{align}
w(t) &= z_{0} \left( 1-\alpha) (1 - t_{y}\right) k_{i}^{\alpha}{\left(t \right)} K_{G}^{1 - \alpha}{\left(t \right)} \\
r(t)+\delta_{k}  &= z_{0}\alpha \left(1 - t_{y}\right) k_{i}{\left(t\right)}^{\alpha-1} K_{G}^{1 - \alpha}{\left(t \right)}
\end{align}

donde $k_i(t) = \frac{K_i(t)}{L_i(t)}$.

Agregando en todas las firmas se obtienen las siguientes relaciones macroeconómicas:

\begin{align}
Y(t) &= Z_0 K(t)^{\alpha} K_G(t)^{1 - \alpha}  \\
w(t) L_0 &= (1 - \alpha)(1 - t_Y) Y(t)  \\
r(t) + \delta_k &= \alpha (1 - t_Y) Z_0 \left( \frac{K_G(t)}{K(t)} \right)^{1 - \alpha}
\end{align}

donde $Y(t) \equiv \sum_i Y_i(t)$ es el producto agregado, $K(t) \equiv \sum_i K_i(t)$, el stock de capital privado agregado, $L_0$, la población agregada (constante e igual a la oferta de trabajo) y $Z_0 \equiv z_0 L_0^{1-\alpha}$ es una constante positiva.

Por otro lado, la identidad de acumulación del capital público es:

\begin{equation}
\dot{K}_G(t) = I_G(t) - \delta_g K_G(t)
\end{equation}

y la restricción presupuestaria estática del gobierno sin impuestos de suma fija queda:

\begin{equation}
t_Y Y(t) = I_G(t) + g Y(t) \iff I_G(t) = (t_y-g)Y(t), \quad g<t_y
\end{equation}


# Problena del Consumidor

Se postula la existencia de una familia representativa que vive para siempre, que maximiza la utilidad de toda la vida:

\begin{equation}
\Lambda = \int\limits_{0}^{\infty} \frac{\left(C^{1 - \frac{1}{\sigma}}{\left(t \right)} - 1\right) e^{- \rho t}}{1 - \frac{1}{\sigma}}\, dt
\end{equation}

sujeto a la identidad de acumulación de activos:
\begin{equation}
\frac{d}{d t} A{\left(t \right)} = L_{0} w{\left(t \right)} - \left(t_{c}{\left(t \right)} + 1\right) C{\left(t \right)} + A{\left(t \right)} r{\left(t \right)} - T{\left(t \right)}
\end{equation}

El hamiltoniano del problema es:

\begin{equation}
H = \frac{\left(C^{1 - \frac{1}{\sigma}}{\left(t \right)} - 1\right) e^{- \rho t}}{1 - \frac{1}{\sigma}} + \lambda{\left(t \right)} \left(L_{0} w{\left(t \right)} + r A{\left(t \right)} - \left(t_{c}{\left(t \right)} + 1\right) C{\left(t \right)} - T{\left(t \right)}\right)
\end{equation}

In [ ]:
# Problema del consumidor
Lambda = sp.Integral(((C**(1-1/sigma)-1)/(1-1/sigma))*sp.exp(-rho*t), (t, 0, sp.oo)) # función objetivo
U = sp.diff(sp.Integral(((C**(1-1/sigma)-1)/(1-1/sigma))*sp.exp(-rho*t), t), t)
lamda = sp.Function('lamda')(t) # multiplicador
H = U + lamda * (r*A + w*L_0 - (1 + t_c)*C - T)

In [ ]:
# CPO consumidor
CPO_consumo = sp.Eq(sp.diff(H, C), 0)
CPO_multiplicador = sp.Eq(sp.diff(lamda, t), -sp.diff(H, A))
RP = sp.Eq(sp.diff(A, t), r*A + w*L_0 - (1+t_c)*C - T) # identidad de acumulación de activos (condición de estado)

In [ ]:
# Euler: Derivar CPO consumo respecto a t
d1 = sp.solve(CPO_consumo, lamda, dict=True)
d11 = sp.diff(d1[0][lamda], t) # esto es igual a lamda punto

In [ ]:
sp.Eq(sp.Derivative(lamda), d11)

Eq(Derivative(lamda(t), t), -rho*C(t)**(-1 + (sigma - 1)/sigma)*exp(-rho*t)/(t_c(t) + 1) + (-1 + (sigma - 1)/sigma)*C(t)**(-1 + (sigma - 1)/sigma)*exp(-rho*t)*Derivative(C(t), t)/((t_c(t) + 1)*C(t)) - C(t)**(-1 + (sigma - 1)/sigma)*exp(-rho*t)*Derivative(t_c(t), t)/(t_c(t) + 1)**2)

In [ ]:
CPO_multiplicador

Eq(Derivative(lamda(t), t), -lamda(t)*r(t))

In [ ]:
lamda_sol = sp.solve(sp.Eq(d11, CPO_multiplicador.rhs), lamda, dict=True) # resuelvo lambda

In [ ]:
CPO_consumo

Eq((-t_c(t) - 1)*lamda(t) + C(t)**(1 - 1/sigma)*exp(-rho*t)/C(t), 0)

In [ ]:
Euler_Consumo = sp.solve(CPO_consumo.subs(lamda, lamda_sol[0][lamda]), C, dict=True) # remplazo lamda y despejo C

In [ ]:
sp.simplify(Euler_Consumo[0][C])

-(t_c(t) + 1)*Derivative(C(t), t)/(sigma*(rho*t_c(t) + rho - r(t)*t_c(t) - r(t) + Derivative(t_c(t), t)))

Finalmente se llega a la ecuación de Euler de consumo:
\begin{equation}
C(t) = - \frac{\left(t_{c}{\left(t \right)} + 1\right) \frac{d}{d t} C{\left(t \right)}}{\sigma \left(\rho t_{c}{\left(t \right)} + \rho - r{\left(t \right)} t_{c}{\left(t \right)} - r{\left(t \right)} + \frac{d}{d t} t_{c}{\left(t \right)}\right)}
\end{equation}
equivalente a
\begin{equation}
\frac{\dot{C}{\left(t \right)}}{C{\left(t \right)}} = \sigma \left(r{\left(t \right)} - \rho  - \frac{\dot{t_c}{\left(t \right)}}{t_{c}{\left(t \right)} + 1}\right)
\end{equation}
donde $\sigma$ no depende del tiempo ya que la función de utilidad implementada tiene elasticidad de sutitución constante.

# Linearización de ecuaciones diferenciales ordinarias bidimensionales

El teorema de existencia y unicidad para problemas de valor inicial garantiza la existencia de una solución única para cierto intervalo de tiempo. Sin embargo, para entender el comportamiento del sistema en el largo plazo, es necesario analizar la **estabilidad** del sistema alrededor de sus puntos fijos.

Consideremos un sistema de dos ecuaciones diferenciales ordinarias acopladas y autónomas de la forma general:

\begin{aligned}
\dot{x} &= f(x,y)\\
\dot{y} &= g(x,y)
\end{aligned}

Supongamos que:
\begin{equation}
\overline{x} = (x^*,y^*)
\end{equation}

es un punto fijo (estado estacionario) del sistema, es decir:

\begin{equation}
f(x^*, y^*) = 0 \quad \text{y} \quad g(x^*, y^*) = 0.
\end{equation}

Ahora describimos la dinámica del sistema en torno al punto fijo, introduciendo perturbaciones pequeñas $u = x - x^* \quad \text{y} \quad  v = y - y^*$, de modo que:

\begin{equation}
x = x^* + u, \quad y = y^* + v.
\end{equation}

Sustituyendo y aplicando una expansión de Taylor de primer orden (descartando términos de orden superior), obtenemos:

\begin{aligned}
\dot{u} &= f(x^* + u, y^* + v) \approx f(x^*, y^*) + u \frac{\partial f}{\partial x} + v \frac{\partial f}{\partial y} \\ &\approx u \frac{\partial f}{\partial x} + v \frac{\partial f}{\partial y} \\
\dot{v} &= g(x^* + u, y^* + v) \approx g(x^*, y^*) + u \frac{\partial g}{\partial x} + v \frac{\partial g}{\partial y} \\ &\approx u \frac{\partial g}{\partial x} + v \frac{\partial g}{\partial y}
\end{aligned}

En forma matricial:

\begin{equation}
\begin{bmatrix}
\dot{u} \\
\dot{v}
\end{bmatrix} \approx
\left. \begin{bmatrix}
\frac{\partial f}{\partial x} & \frac{\partial f}{\partial y} \\
\frac{\partial g}{\partial x} & \frac{\partial g}{\partial y}
\end{bmatrix} \right|_{(x^*, y^*)}
\begin{bmatrix}
x - x^* \\
y - y^*
\end{bmatrix}
\end{equation}

La matriz que aparece en el segundo miembro es la matriz jacobiana evaluada en el punto fijo. Sus autovalores determinan la estabilidad del sistema: indican si las perturbaciones iniciales crecen, decrecen u oscilan con el tiempo. A continuación, se obtiene la matriz del modelo con infraestructura pública, para evaluar los signos de sus autovalores y determinar la estabilidad del sistema.


In [ ]:
# Tabla 14.2
T21 = sp.Eq(sp.Derivative(C)/C, sigma*(r-rho))
T22 = sp.Eq(sp.Derivative(K)/K, (1-t_y)*Z_0*((K/K_G))**(alpha-1)-C/K-delta_k)
T23 = sp.Eq(sp.Derivative(K_G)/K_G, (t_y-g)*Z_0*((K/K_G))**(alpha)-delta_g)
c = sp.Eq(r, alpha*(1-t_y)*Z_0*(k)**(alpha-1)-delta_k) # T24

Las cuatro ecuaciones que caracterizan al modelo de creciemiento $AK$ con capital público son:

\begin{align}
\frac{\frac{d}{d t} C{\left(t \right)}}{C{\left(t \right)}} &= \sigma \left(r(t) - \rho\right) \tag{1} \\ \frac{\frac{d}{d t} K{\left(t \right)}}{K{\left(t \right)}} &= Z_{0} \left(\frac{K{\left(t \right)}}{K_{G}{\left(t \right)}}\right)^{\alpha - 1} \left(1 - t_{y}\right) - \delta_{k} - \frac{C{\left(t \right)}}{K{\left(t \right)}} \tag{2} \\ \frac{\frac{d}{d t} K_{G}{\left(t \right)}}{K_{G}{\left(t \right)}} &= \left(t_{y}-g\right) Z_{0} \left(\frac{K{\left(t \right)}}{K_{G}{\left(t \right)}}\right)^{\alpha} - \delta_{g} \tag{3} \\ r{\left(t \right)} &= Z_{0} \alpha \left(1 - t_{y}\right) k^{\alpha - 1}{\left(t \right)} - \delta_{k} \tag{4}
\end{align}

donde:
*   En (1), la ecuación de Euler de consumo, se supone que el impuesto al ingreso es constante en el tiempo.
*   La ecuación (2) resulta del producto agregado obtenido en el problema de la firma, la ecuación de moviemiento del capital privado, la restricción presupuestaria del gobierno estática y la identidad del ingreso nacional: $Y(t) = C(t) + I(t) +gY(t)+I_G(t)$.
*   La ecuación (3) se obtiene a partir de la ecuación de moviemiento del capital público, el producto agregado y la restricción presupuestaria del gobierno estática.

Restando en ambos miembros 2 a 1 y 3 a 2, se rescribe el modelo.

In [ ]:
# Intermezzo 14.1
# T21 - T22
a0 = sp.Eq(T21.lhs - T22.lhs, T21.rhs - T22.rhs)

In [ ]:
(a0.subs(C/K, theta)).subs(K/K_G, k)

Eq(-Derivative(K(t), t)/K(t) + Derivative(C(t), t)/C(t), -Z_0*(1 - t_y)*k(t)**(alpha - 1) + delta_k + sigma*(-rho + r(t)) + theta(t))

In [ ]:
sp.Eq(sp.diff(theta)/theta, sp.diff(sp.log(C/K),t)) # por definición de derivada del logaritmo

Eq(Derivative(theta(t), t)/theta(t), (-C(t)*Derivative(K(t), t)/K(t)**2 + Derivative(C(t), t)/K(t))*K(t)/C(t))

In [ ]:
a = sp.Eq(sp.Derivative(theta)/theta,((a0.subs(C/K, theta)).subs(K/K_G, k)).rhs);a

Eq(Derivative(theta(t), t)/theta(t), -Z_0*(1 - t_y)*k(t)**(alpha - 1) + delta_k + sigma*(-rho + r(t)) + theta(t))

In [ ]:
# T22 - T23
b0 = sp.Eq(T22.lhs - T23.lhs, T22.rhs - T23.rhs)

In [ ]:
(b0.subs(C/K, theta)).subs(K/K_G, k)

Eq(-Derivative(K_G(t), t)/K_G(t) + Derivative(K(t), t)/K(t), Z_0*(1 - t_y)*k(t)**(alpha - 1) - Z_0*(-g + t_y)*k(t)**alpha + delta_g - delta_k - theta(t))

In [ ]:
sp.Eq(sp.Derivative(k)/k, sp.diff(sp.log(K/K_G),t)) # por definición de derivada del logaritmo

Eq(Derivative(k(t), t)/k(t), (-K(t)*Derivative(K_G(t), t)/K_G(t)**2 + Derivative(K(t), t)/K_G(t))*K_G(t)/K(t))

In [ ]:
b = sp.Eq(sp.Derivative(k)/k,((b0.subs(C/K, theta)).subs(K/K_G, k)).rhs);b

Eq(Derivative(k(t), t)/k(t), Z_0*(1 - t_y)*k(t)**(alpha - 1) - Z_0*(-g + t_y)*k(t)**alpha + delta_g - delta_k - theta(t))

El sistema queda:

\begin{align}
\frac{\partial \ln \theta{\left(t \right)}}{\partial t} &= - Z_{0} \left(1 - t_{y}\right) k^{\alpha - 1}{\left(t \right)} + \delta_{k} + \sigma \left(r{\left(t \right)}- \rho \right) + \theta{\left(t \right)}
\tag{a} \\ \frac{\partial \ln k{\left(t \right)}}{\partial t} &= Z_{0} \left(1 - t_{y}\right) k^{\alpha - 1}{\left(t \right)} - Z_{0} \left(t_{y} - g\right) k^{\alpha}{\left(t \right)} + \delta_{g} - \delta_{k} - \theta{\left(t \right)} \tag{b} \\ r{\left(t \right)} &= Z_{0} \alpha \left(1 - t_{y}\right) k^{\alpha - 1}{\left(t \right)} - \delta_{k} \tag{c}
\end{align}

donde $k(t) = \frac{K(t)}{K_G(t}$ y se utilizó $d \ln x(t)/dt = \dot{x}(t)/x(t)$.

Si se define una variable auxiliar $\tilde{x}(t)$:

\begin{align}
a \tilde{x}(t) &= \ln \left( \frac{x(t)}{x^*} \right)^\alpha \iff \left( \frac{x(t)}{x^*} \right)^\alpha = e^{a \tilde{x}(t)} \tag{d}
\end{align}

donde $x^*$ es el valor de estado estacionario para $x(t)$ y $a$ un escalar. Dado que $x(t)$ está cerca de su valor de estado estacionario ($x(t)/x^* \approx 1$ y $\tilde{x}(t) \approx 0$), se tiene que:

\begin{align}
\left( \frac{x(t)}{x^*} \right)^a \approx 1 + a \tilde{x}(t) \iff a \tilde{x}(t) = \left( \frac{x(t)}{x^*} \right)^a - 1 \tag{e}
\end{align}

Respecto a la ecuación (a), en estado estacioanario, $d \ln \theta(t)/dt = 0$, así que:

\begin{align}
0 = \sigma [r^* - \rho] - (1 - t_Y) Z_0 (k^*)^{\alpha - 1} + \theta^* + \delta_k \tag{f}
\end{align}

Restando (f) de (a) se obtiene:

\begin{align}
\frac{d \ln \theta(t)}{dt}
&= \sigma [r(t) - r^*] - (1 - t_Y) Z_0 \left[k(t)^{\alpha - 1} - (k^*)^{\alpha - 1} \right] + \theta(t) - \theta^* \\
&= \sigma r^* \left( \frac{r(t)}{r^*} - 1 \right) - (1 - t_Y) Z_0 (k^*)^{\alpha - 1} \left( \left( \frac{k(t)}{k^*} \right)^{\alpha - 1} - 1 \right) + \theta^* \left( \frac{\theta(t)}{\theta^*} - 1 \right) \\
&= \sigma r^* \tilde{r}(t) - (\alpha - 1) \frac{r^* + \delta_k}{\alpha} \tilde{k}(t) + \theta^* \tilde{\theta}(t) \tag{g}
\end{align}

donde usamos (e) (dos veces con $\alpha = 1$ y una vez con $\alpha = \alpha - 1$), y que según (c) se tiene: $(1 - t_Y) Z_0 (k^*)^{\alpha - 1} = \frac{r^* + \delta_k}{\alpha}$. Y por la definción de $\tilde{x}(t)$, podemos escribir (g) como:

\begin{align}
\frac{d \ln \theta(t)}{dt}
= \sigma r^* \ln \left( \frac{r(t)}{r^*} \right)
+ (1 - \alpha) \frac{r^* + \delta_k}{\alpha} \ln \left( \frac{k(t)}{k^*} \right)
+ \theta^* \ln \left( \frac{\theta(t)}{\theta^*} \right) \tag{h}
\end{align}

Aplicando el mismo procedimiento a la ecuación (b):

\begin{align}
\frac{d \ln k(t)}{dt}
&= (1 - t_Y) Z_0 (k^*)^{\alpha - 1} \left( \left( \frac{k(t)}{k^*} \right)^{\alpha - 1} - 1 \right) \\
&\quad - (t_Y - g) Z_0 (k^*)^{\alpha} \left( \left( \frac{k(t)}{k^*} \right)^{\alpha} - 1 \right) - \theta^* \left( \frac{\theta(t)}{\theta^*} - 1 \right) \\
&= - \left[ (1 - \alpha) \frac{r^* + \delta_k}{\alpha} + \alpha (\gamma^* + \delta_g) \right] \ln \left( \frac{k(t)}{k^*} \right) - \theta^* \ln \left( \frac{\theta(t)}{\theta^*} \right) \tag{i}
\end{align}

donde usamos que: $(t_Y - g) Z_0 (k^*)^{\alpha} = \gamma^* + \delta_g$

Finalmente, a partir de la ecuación (c):

\begin{align}
r^* \left( \frac{r(t)}{r^*} - 1 \right)
&= \alpha (1 - t_Y) Z_0 (k^*)^{\alpha - 1} \left( \left( \frac{k(t)}{k^*} \right)^{\alpha - 1} - 1 \right) \\
r^* \ln \left( \frac{r(t)}{r^*} \right)
&= (r^* + \delta_k)(\alpha - 1) \ln \left( \frac{k(t)}{k^*} \right) \tag{j}
\end{align}

Sustituyendo (j) en (h) se obtiene:

\begin{align}
\frac{d \ln \theta(t)}{dt}
&= \sigma (r^* + \delta_k)(\alpha - 1) \ln \left( \frac{k(t)}{k^*} \right)
+ (1 - \alpha) \frac{r^* + \delta_k}{\alpha} \ln \left( \frac{k(t)}{k^*} \right)
+ \theta^* \ln \left( \frac{\theta(t)}{\theta^*} \right)\\
\frac{d \ln \theta(t)}{dt} &= \theta^* \ln \left( \frac{\theta(t)}{\theta^*} \right) + \frac{(1-\alpha)(1-\alpha\sigma)(r^*+\delta_k)}{\alpha} \ln \left( \frac{k(t)}{k^*} \right) \tag{k}
\end{align}


En términos matriciales:

\begin{equation}
\begin{bmatrix}
\frac{d \ln \theta(t)}{dt} \\
\frac{d \ln k(t)}{dt}
\end{bmatrix}
=
\left. \Delta \right|_{(\theta^*, k^*)}
\begin{bmatrix}
\ln  \theta(t) - \ln \theta^* \\
\ln k(t) - \ln k^*
\end{bmatrix}
\end{equation}
donde
\begin{equation}
\Delta=
\begin{bmatrix}
\theta^{\ast} & \frac{(1-\alpha)(1-\alpha\sigma)(r^*+\delta_k)}{\alpha} \\
- \theta^{\ast} & -\frac{(1 - \alpha) (r^* + \delta_k) + \alpha^2 (\gamma^* + \delta_g)}{\alpha}
\end{bmatrix}
\end{equation}

Donde la priemera fila del sistema es la ecuación $(k)$ y la segunda, la ecuación $(i)$. El determinante de $\Delta$ está dado por:

\begin{equation}
|\Delta| = -\theta^*
\left[ (1 - \alpha)\sigma(r^* + \delta_k) + \alpha \right] \left( \gamma^* + \delta_g \right) < 0
\end{equation}

Por lo tanto, el producto de las raíces características de $\Delta$ es negativo, es decir, hay una raíz negativa (estable) y una raíz positiva (inestable), por lo que el modelo sigue una trayectoria de silla.

Recordemos que ambos stocks de capital son variables predeterminadas (no saltan), mientras que el consumo privado es una variable de salto. Por lo tanto, $\theta(t)$ es una variable de salto (porque $C(t)$ lo es), mientras que $k(t)$ es una variable predeterminada (porque lo son $K(t)$ y $K_G(t)$). Dado un valor inicial $K(0)$ y $K_G(0)$ (y por lo tanto $k(0) = K(0)/K_G(0)$), el modelo converge a lo largo del sendero de silla hacia el equilibrio de estado estacionario. La velocidad de transición está dada por el valor absoluto de la raíz estable.


In [ ]:
# Definición de estado estacionario y funciones auxiliares
theta_ee = sp.Symbol(f'theta^*') # estado estacionario
k_ee = sp.Symbol(f'k^*') # estado estacionario
r_ee = sp.Symbol(f'r^*') # estado estacionario
r_moño = sp.Function(r'\tilde{r}')(t) # auxiliar
k_moño = sp.Function(r'\tilde{k}')(t) # auxiliar
theta_moño = sp.Function(r'\tilde{\theta}')(t) # auxiliar

In [ ]:
# Símbolos (se olvida?)
delta_k = sp.symbols('delta_k', domain=(0,1)) # depreciación del capital privado
delta_g = sp.symbols('delta_g', domain=(0,1)) # depreciación del capital publico
# Definición de proporciones (linearización)
g = sp.symbols('g', real=True, domain=(0,t_y)) # participación del consumo público en el ingreso nacional
global_assumptions.add(g > 0)
global_assumptions.add(g < t_y)
theta = sp.Function('theta')(t) # proporción consumo-capital privado: theta = sp.Eq(theta, C/K)
k = sp.Function('k')(t) # proporción capital privado-capital publico: k = sp.Eq(k, K/K_G)
gamma_ee = sp.symbols('gamma^*', real=True)

In [ ]:
## Restar estado estacionario y usar funciones auxiliares
# Primero theta
f = sp.Eq(0, sigma*(r_ee - rho)-(1-t_y)*Z_0*(k_ee)**(alpha-1)+delta_k+theta_ee) # estado estacionario de (a)

In [ ]:
g0 = sp.solve([sp.Eq(a.lhs - f.lhs, a.rhs - f.rhs), sp.Eq(r_moño, r/r_ee-1), sp.Eq(theta_moño, theta/theta_ee-1)],
         [sp.Derivative(theta)/theta,r,theta], dict=True)[0][sp.Derivative(theta)/theta]

In [ ]:
sp.Eq(sp.Derivative(theta)/theta, g0)

Eq(Derivative(theta(t), t)/theta(t), -Z_0*k^***(alpha - 1)*t_y + Z_0*k^***(alpha - 1) + Z_0*t_y*k(t)**(alpha - 1) - Z_0*k(t)**(alpha - 1) + r^**sigma*\tilde{r}(t) + theta^**\tilde{\theta}(t))

In [ ]:
g1 = sp.Eq(sp.Derivative(theta)/theta,(1-t_y)*(-Z_0)*k_ee**(alpha-1)*((k/k_ee)**(alpha-1)-1)+r_ee*sigma*r_moño + theta_ee*theta_moño);g1 # factor común

Eq(Derivative(theta(t), t)/theta(t), -Z_0*k^***(alpha - 1)*(1 - t_y)*((k(t)/k^*)**(alpha - 1) - 1) + r^**sigma*\tilde{r}(t) + theta^**\tilde{\theta}(t))

In [ ]:
g2 = g1.subs((k/k_ee)**(alpha-1)-1,(alpha-1)*k_moño);g2 # función auxiliar de k

Eq(Derivative(theta(t), t)/theta(t), -Z_0*k^***(alpha - 1)*(1 - t_y)*(alpha - 1)*\tilde{k}(t) + r^**sigma*\tilde{r}(t) + theta^**\tilde{\theta}(t))

In [ ]:
g = g2.subs((1-t_y)*Z_0*k_ee**(alpha-1),(r_ee+delta_k)/alpha);g # estado estacionario de r (c)

Eq(Derivative(theta(t), t)/theta(t), r^**sigma*\tilde{r}(t) + theta^**\tilde{\theta}(t) - (alpha - 1)*(delta_k + r^*)*\tilde{k}(t)/alpha)

In [ ]:
h = (g.subs(k_moño, sp.ln(k/k_ee))).subs(theta_moño,sp.ln(theta/theta_ee));h # def k_moño, theta_moño

Eq(Derivative(theta(t), t)/theta(t), r^**sigma*\tilde{r}(t) + theta^**log(theta(t)/theta^*) - (alpha - 1)*(delta_k + r^*)*log(k(t)/k^*)/alpha)

In [ ]:
# Último paso antes de sustituir en h y obtener primera fila de la matriz
c_ee = sp.Eq(r_ee, Z_0*alpha*(1-t_y)*k_ee**(alpha-1)-delta_k);c_ee # estado estacionario de (c)

Eq(r^*, Z_0*alpha*k^***(alpha - 1)*(1 - t_y) - delta_k)

In [ ]:
j0 = sp.Eq(c.lhs - c_ee.lhs, c.rhs - c_ee.rhs).subs(r-r_ee,r_ee*sp.log(r/r_ee));j0 # c - c en ee

Eq(r^**log(r(t)/r^*), -Z_0*alpha*k^***(alpha - 1)*(1 - t_y) + Z_0*alpha*(1 - t_y)*k(t)**(alpha - 1))

In [ ]:
j1 = sp.Eq(j0.lhs, ((k/k_ee)**(alpha-1)-1)*Z_0*alpha*(1-t_y)*k_ee**(alpha-1));j1 # factor comun (multiplicar y dividir por k_ee**(alhpa-1))

Eq(r^**log(r(t)/r^*), Z_0*alpha*k^***(alpha - 1)*(1 - t_y)*((k(t)/k^*)**(alpha - 1) - 1))

In [ ]:
j2 = j1.subs((k / k_ee)**(alpha - 1) - 1, (alpha - 1) * sp.log(k / k_ee)); j2 # def k_moño para alpha - 1

Eq(r^**log(r(t)/r^*), Z_0*alpha*k^***(alpha - 1)*(1 - t_y)*(alpha - 1)*log(k(t)/k^*))

In [ ]:
j = j2.subs(Z_0*(1-t_y)*k_ee**(alpha-1),(r+delta_k)/alpha);j # c en ee

Eq(r^**log(r(t)/r^*), (alpha - 1)*(delta_k + r(t))*log(k(t)/k^*))

In [ ]:
h

Eq(Derivative(theta(t), t)/theta(t), r^**sigma*\tilde{r}(t) + theta^**log(theta(t)/theta^*) - (alpha - 1)*(delta_k + r^*)*log(k(t)/k^*)/alpha)

In [ ]:
k0 = h.subs(r_ee*r_moño, j.rhs);k0 # j en h

Eq(Derivative(theta(t), t)/theta(t), sigma*(alpha - 1)*(delta_k + r(t))*log(k(t)/k^*) + theta^**log(theta(t)/theta^*) - (alpha - 1)*(delta_k + r^*)*log(k(t)/k^*)/alpha)

In [ ]:
sp.simplify(alpha*sigma*(alpha-1)+(1-alpha))

alpha*sigma*(alpha - 1) - alpha + 1

In [ ]:
1-alpha+sigma*alpha**2-sigma*alpha

alpha**2*sigma - alpha*sigma - alpha + 1

In [ ]:
k_expr = sp.Eq(k0.lhs, theta_ee*sp.log(theta/theta_ee)+((1-alpha)*(1-alpha*sigma)*(r_ee+delta_k))/alpha*sp.ln(k/k_ee));k_expr # factor común

Eq(Derivative(theta(t), t)/theta(t), theta^**log(theta(t)/theta^*) + (1 - alpha)*(delta_k + r^*)*(-alpha*sigma + 1)*log(k(t)/k^*)/alpha)

In [ ]:
# Símbolos (se olvida?)
delta_k = sp.symbols('delta_k', domain=(0,1)) # depreciación del capital privado
delta_g = sp.symbols('delta_g', domain=(0,1)) # depreciación del capital publico
# Definición de proporciones (linearización)
g = sp.symbols('g', real=True, domain=(0,t_y)) # participación del consumo público en el ingreso nacional
global_assumptions.add(g > 0)
global_assumptions.add(g < t_y)
theta = sp.Function('theta')(t) # proporción consumo-capital privado: theta = sp.Eq(theta, C/K)
k = sp.Function('k')(t) # proporción capital privado-capital publico: k = sp.Eq(k, K/K_G)
gamma_ee = sp.symbols('gamma^*', real=True)

In [ ]:
# Segundo k
fb = sp.Eq(0, Z_0*(1-t_y)*k_ee**(alpha-1)-Z_0*(t_y-g)*k_ee**(alpha)+delta_g-delta_k-theta_ee) # estado estacionario de (b)

In [ ]:
g0b = sp.solve([sp.Eq(b.lhs - fb.lhs, b.rhs - fb.rhs), sp.Eq(theta_moño, theta/theta_ee-1)],
         [sp.Derivative(k)/k,theta], dict=True)[0][sp.Derivative(k)/k];g0b

-Z_0*g*k^***alpha + Z_0*g*k(t)**alpha + Z_0*k^***alpha*t_y + Z_0*k^***(alpha - 1)*t_y - Z_0*k^***(alpha - 1) - Z_0*t_y*k(t)**alpha - Z_0*t_y*k(t)**(alpha - 1) + Z_0*k(t)**(alpha - 1) - theta^**\tilde{\theta}(t)

In [ ]:
# factor común
g1b = sp.Eq(sp.Derivative(k)/k, (1-t_y)*Z_0*k_ee**(alpha-1)*((k/k_ee)**(alpha-1)-1)-(t_y-g)*Z_0*k_ee**(alpha)*((k/k_ee)**(alpha)-1)-theta_ee*theta_moño);g1b

Eq(Derivative(k(t), t)/k(t), -Z_0*k^***alpha*(-g + t_y)*((k(t)/k^*)**alpha - 1) + Z_0*k^***(alpha - 1)*(1 - t_y)*((k(t)/k^*)**(alpha - 1) - 1) - theta^**\tilde{\theta}(t))

In [ ]:
g2b = (g1b.replace((k/k_ee)**(alpha-1)-1,(alpha-1)*k_moño)).replace((k/k_ee)**alpha-1,alpha*k_moño);g2b # k moño para alpha y alpha-1

Eq(Derivative(k(t), t)/k(t), -Z_0*alpha*k^***alpha*(-g + t_y)*\tilde{k}(t) + Z_0*k^***(alpha - 1)*(1 - t_y)*(alpha - 1)*\tilde{k}(t) - theta^**\tilde{\theta}(t))

In [ ]:
(k_moño*-((1-alpha)*(1-t_y)*Z_0*k_ee**(alpha-1)+alpha*Z_0*k_ee**alpha*(t_y-g))-theta_ee*theta_moño)

-theta^**\tilde{\theta}(t) + (-Z_0*alpha*k^***alpha*(-g + t_y) - Z_0*k^***(alpha - 1)*(1 - alpha)*(1 - t_y))*\tilde{k}(t)

In [ ]:
# factor común k_moño, estado estacionario de r (c) y gamma ¿crecimiento ee?
i0 = ((k_moño*-((1-alpha)*(1-t_y)*Z_0*k_ee**(alpha-1)+alpha*Z_0*k_ee**alpha*(t_y-g))-theta_ee*theta_moño).subs((1-t_y)*Z_0*k_ee**(alpha-1),(r_ee+delta_k)/alpha)).subs(Z_0*k_ee**alpha*(t_y-g), gamma_ee + delta_g)
i0

-theta^**\tilde{\theta}(t) + (-alpha*(delta_g + gamma^*) - (1 - alpha)*(delta_k + r^*)/alpha)*\tilde{k}(t)

In [ ]:
# definicione k_moño y theta_moño
i = sp.Eq(g2b.lhs, (i0.subs(k_moño, sp.ln(k/k_ee))).subs(theta_moño,sp.ln(theta/theta_ee)));i

Eq(Derivative(k(t), t)/k(t), -theta^**log(theta(t)/theta^*) + (-alpha*(delta_g + gamma^*) - (1 - alpha)*(delta_k + r^*)/alpha)*log(k(t)/k^*))

In [ ]:
k_expr

Eq(Derivative(theta(t), t)/theta(t), theta^**log(theta(t)/theta^*) + (1 - alpha)*(delta_k + r^*)*(-alpha*sigma + 1)*log(k(t)/k^*)/alpha)

In [ ]:
# Expresión matricial
vector_derivadas = sp.Matrix([k_expr.lhs, i.lhs])

In [ ]:
Delta = sp.Matrix([[k_expr.rhs.coeff(sp.log(theta/theta_ee)), k_expr.rhs.coeff(sp.log(k/k_ee))],
 [i.rhs.coeff(sp.log(theta/theta_ee)), i.rhs.coeff(sp.log(k/k_ee))]])

In [ ]:
vector_variaciones = sp.Matrix([sp.log(theta)-sp.log(theta_ee), sp.log(k)-sp.log(k_ee)])

In [ ]:
Sistema = sp.Eq(vector_derivadas, sp.MatMul(Delta, vector_variaciones, evaluate=False));Sistema

Eq(Matrix([
[Derivative(theta(t), t)/theta(t)],
[        Derivative(k(t), t)/k(t)]]), Matrix([
[ theta^*,           (1 - alpha)*(delta_k + r^*)*(-alpha*sigma + 1)/alpha],
[-theta^*, -alpha*(delta_g + gamma^*) - (1 - alpha)*(delta_k + r^*)/alpha]])*Matrix([
[-log(theta^*) + log(theta(t))],
[        -log(k^*) + log(k(t))]]))

In [ ]:
det_Delta = sp.simplify(Delta.det());det_Delta

theta^**(-alpha*delta_g + alpha*delta_k*sigma - alpha*gamma^* + alpha*r^**sigma - delta_k*sigma - r^**sigma)

In [ ]:
# falta ver el signo